# CorrDiff

## 1. 模型简介
CorrDiff是一种利用高分辨率天气数据和更粗略的ERA5再分析数据训练了一种经济高效的随机降尺度模型，采用 UNet 和扩散的两步法来解决多尺度挑战，在预测极端天气和准确捕捉强降雨和台风动态等多变量关系方面表现出色，为全球到公里级的机器学习天气预报带来了光明的未来。[CorrDiff paper](https://arxiv.org/abs/2309.15214)


## 1.1 模型结构
CorrDiff将生成过程分解为两步：首先使用25公里分辨率的天气数据，第一步使用UNet回归预测得到条件均值μ，第二步通过EDM扩散模型用以修正均值的偏差（即学习残差r的分布，r基本上是零均值并且相比于目标数据x的分布表现出较小的分布偏移，即相比于x的方差更小，因此可以在扩散过程中使用较小的噪声水平进行训练，得到的r用于修正均值μ的偏差），二者共同构成概率性的高分辨率的区域预报。

<img src=../../../doc/corrdiff_illustration.png width=500 height=300 />



## 2. 软件环境准备

### 2.1 基于dtk适配软件
<div class="alert alert-warning"> WARNING：镜像中环境已配置，此步骤省略 </div>
<p>1. 基础软件环境DTK：推荐环境 dtk=dtk-24.04.2  下载链接：<a href="https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04" target="_blank">https://cancon.hpccube.com:65024/1/main/DTK-24.04.2/Ubuntu22.04 </a> </p>
<p>2. pytorch软件包下载：推荐环境 torch=2.1.0 py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/pytorch/DAS1.2" target="_blank">https://download.sourcefind.cn:65024/4/main/pytorch/DAS1.2 </a> </p>
<p>3. torchvision软件包下载：推荐环境 torchvision=0.16.0 py310 下载链接: <a href="https://download.sourcefind.cn:65024/4/main/vision/DAS1.2" target="_blank">https://download.sourcefind.cn:65024/4/main/vision/DAS1.2 </a> </p>

### 2.2 软件环境检查

In [ ]:
# 检查torch版本
import torch
import os
import onescience

version = torch.__version__
num = float(version[:3])
# assert num == 1.10

# 检查硬件环境
device = "cpu"
if os.system('rocm-smi 2>/dev/null || hy-smi 2>/dev/null')==0:
    device = "dtk"

elif os.system('nvidia-smi 2>/dev/null')==0: 
    device = "cuda"

print("torch version:", version)
print("onescience version:", onescience.__version__)
print("device =", device)

### 2.3 软件依赖安装


<div class="alert alert-warning"> WARNING: 镜像中环境已配置，此步骤省略 </div>

<div class="alert alert-note" style="color: blue;">
Note: 检查镜像环境下onescience版本号是否与当前目录onescience安装包版本一致，若不一致则需要卸载当前环境下的 onescience 包，并安装当前目录的 whl 包。安装指令参考：
<pre><code>pip uninstall onescience&&pip install &lt;onescience安装包名称&gt;.whl</code></pre>
</div>

## 3. 素材准备
### 3.1 数据集准备


CorrDiff训练在官方提供的台湾数据集上进行演示，以 [ERA5 数据集](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) 为条件。可从 [https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_datasets_cwa](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_datasets_cwa)下载</p>请确保当前项目中包含dataset目录且结构如下：
```
 ├── XLAT (450, 450) float32
 ├── XLAT_U (450, 451) float32
 ├── XLAT_V (451, 450) float32
 ├── XLONG (450, 450) float32
 ├── XLONG_U (450, 451) float32
 ├── XLONG_V (451, 450) float32
 ├── XTIME () float32
 ├── cwb (35064, 4, 450, 450) float32
 ├── cwb_center (4,) float32
 ├── cwb_pressure (4,) float64
 ├── cwb_scale (4,) float32
 ├── cwb_valid (35064,) int8
 ├── cwb_variable (4,) <U26
 ├── era5 (35064, 20, 450, 450) float32
 ├── era5_center (20,) float32
 ├── era5_pressure (20,) float64
 ├── era5_scale (20,) float32
 ├── era5_valid (35064, 20) int8
 ├── era5_variable (20,) <U19
 └── time (35064,) int64
```

主要目录结构如下
```
corrdiff-torch\
|----conf\
|    |----dataset\
|    |----generation\
|    |----generation\
|    |----model\
|    |----references\
|    |----sampler\
|    |----training\
|    |----validation\
|    |----config_generate_mini.yaml
|    |----config_generate.yaml
|    |----config_training_mini_diffusion.yaml
|    |----config_training_mini_regression.yaml
|    |----config_training.yaml
|----datasets\
|    |----__init__.py
|    |----base.py
|    |----cwb.py
|    |----dataset.py
|    |----hrrrmini.py
|    |----img_utils.py
|    |----norm.py
|----onescience-0.1.0-py3-none-any.whl
|----README.md
|----corrdiff-pytorch.ipynb
|----dataset_info.txt
|----generate.py
|----score_samples.py
|----train.py
```

## 4 训练
### 4.1 单卡训练
模型训练配置在./conf/training文件夹，数据集、模型和训练参数配置分别加载如下：
- dataset/cwb_train：数据集使用台湾数据集配置；
- model/corrdiff_regression: 模型结构加载回归模型或者扩散模型；
- training/corrdiff_regression：不同类型模型训练配置。

In [ ]:
# 执行训练
!python train.py

### 4.2 单节点多卡训练

In [ ]:
# 执行训练
!torchrun --standalone --nnodes=1 --nproc_per_node=4 train.py

## 5. 相关文献和引用
- [Residual Diffusion Modeling for Km-scale Atmospheric Downscaling](https://arxiv.org/pdf/2309.15214.pdf)
